In [2]:
import sys

sys.path.insert(0, '/home/jupyter/AdversarialRobustness')
sys.path.insert(0, '/home/jupyter/AdversarialRobustness/vim')

In [2]:
#!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118
#!pip install -r AdversarialRobustness/vim_requirements.txt
#!pip install causal-conv1d==1.1.0
#!pip install mamba-ssm==1.1.1
#!pip install timm
#!pip install -r AdversarialRobustness/requirements.txt

In [3]:
# Add sbin to path for Trigon - fix of common Debian error
import os
original_path = os.environ.get('PATH')
os.environ['PATH'] = original_path + ':/sbin'

In [4]:
#import mamba_ssm
#mamba_ssm.__file__

In [5]:
# Replace pip mamba_ssm with author modification
#!rm -rf /opt/conda/envs/pytorch/lib/python3.10/site-packages/mamba_ssm
#!cp -r ./mamba_ssm /opt/conda/envs/pytorch/lib/python3.10/site-packages/mamba_ssm

In [4]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import requests
from matplotlib import pyplot as plt
from torchvision import datasets
from transformers import ViTImageProcessor, ViTForImageClassification

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

import pickle

import loadVim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Load util functions to generate adversarial dataset
from captum.robust import FGSM, MinParamPerturbation
from genAdvDataset import fgsmAttack, calcImageDistanceMetrics, saveImageTensor, genImageNetteConvDict

# Generate ImageNette Adversarial Dataset

In [ ]:
# Use ViT for config, as Vim doesn't contain metadata
confproxymod = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
netteToImageNetConv = genImageNetteConvDict(confproxymod)

In [5]:
# Load Model and prepare processor
model, processor = loadVim.loadVim()
normalize = transforms.Normalize(mean=processor.image_mean, std=processor.image_std)

# Predict function
def pred(inputs, model, preprocess=True):
    # preprocess the image
    if preprocess:
        inputs = normalize(inputs)

    # If input is not batched, batch it
    if len(inputs.shape) < 4:
        inputs = inputs.unsqueeze(0)

    outputs = model(inputs.to(device))
    logits = outputs.logits
    val, idx = torch.max(logits, dim=1)
    return idx

# Load dataset
def process_image(image):
    return processor(image, return_tensors="pt").pixel_values.squeeze(0)

data = datasets.ImageFolder('./AdversarialRobustness/imagenette/val', transform=transforms.Lambda(process_image))

dataLoader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False, num_workers=0)

# Adversarial attack
# Min Param Perturbation works only with batch size 1 for our purposes
min_pert = MinParamPerturbation(
                                forward_func = lambda *args, **kwargs: model(*args, **kwargs).logits,
                                attack = fgsmAttack,
                                arg_name = 'epsilon',
                                mode = "binary",
                                arg_min = 0.001,
                                arg_max = 1.5,
                                arg_step = 0.001,
                                preproc_fn = normalize,
                                apply_before_preproc=True
                            )

alt_im, min_eps, inputs = None, None, None

path_adversarial_dataset = './AdversarialRobustness/data_adv_vim'
os.makedirs(path_adversarial_dataset, exist_ok=True)

log = []

for i, (input, label) in enumerate(dataLoader):
    input = input.to(device)
    # Convert label item to ImageNet label
    label = torch.tensor([netteToImageNetConv[label.item()]]).to(device)
    label = label.to(device)

    prediction = pred(input, model)

    if prediction == label:
        # We only care for adversarial examples in the case when the model could correctly predict otherwise
        attack_kwargs={'model':model,'true_label':label}

        alt_im, min_eps = min_pert.evaluate(input, attack_kwargs=attack_kwargs, target=label)

        altpred = pred(alt_im, model)
        print(f"adv_{i} is a {model.config.id2label[label.item()]} but classifies as {model.config.id2label[altpred.item()]} with epsilon {min_eps}")

        mse, ssim_val, linf = calcImageDistanceMetrics(input, alt_im)
        log.append({"index": i, "label": label.item(), "prediction": altpred.item(), "epsilon": min_eps, "mse": mse, "ssim": ssim_val, "linf": linf})

        # Save the adversarial example to new dataset
        saveImageTensor(alt_im, path_adversarial_dataset, label, f"adv_{i}")
    else:
        print(f"Incorrect classification, label: {label}, prediction: {prediction}, skipping...")

# Pickle the log
with open(f"{path_adversarial_dataset}/log.pkl", "wb") as f:
    pickle.dump(log, f)

FileNotFoundError: [Errno 2] No such file or directory: './AdversarialRobustness/imagenette/val'

# Generate Celeb Adversarial Dataset

In [6]:
def genAdvUtil(model, processor, path_adversarial_dataset, path_original_dataset):
    normalize = transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
    os.makedirs(path_adversarial_dataset, exist_ok=True)
    
    # Predict function
    def pred(inputs, model, preprocess=True):
        # preprocess the image
        if preprocess:
            inputs = normalize(inputs)

        # If input is not batched, batch it
        if len(inputs.shape) < 4:
            inputs = inputs.unsqueeze(0)

        outputs = model(inputs.to(device))
        logits = outputs.logits
        val, idx = torch.max(logits, dim=1)
        return idx

    # Load dataset
    def process_image(image):
        return processor(image, return_tensors="pt").pixel_values.squeeze(0)

    data = datasets.ImageFolder(path_original_dataset, transform=transforms.Lambda(process_image))

    dataLoader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False, num_workers=0)

    # Adversarial attack
    # Min Param Perturbation works only with batch size 1 for our purposes
    min_pert = MinParamPerturbation(
                                    forward_func = lambda *args, **kwargs: model(*args, **kwargs).logits,
                                    attack = fgsmAttack,
                                    arg_name = 'epsilon',
                                    mode = "binary",
                                    arg_min = 0.001,
                                    arg_max = 3,
                                    arg_step = 0.001,
                                    preproc_fn = normalize,
                                    apply_before_preproc=True
                                )

    alt_im, min_eps, inputs = None, None, None
    
    log = []
    advFailCount = 0

    for i, (input, label) in enumerate(dataLoader):
        input = input.to(device)
        label = label.to(device)

        prediction = pred(input, model)

        if prediction == label:
            # We only care for adversarial examples in the case when the model could correctly predict otherwise
            attack_kwargs={'model':model,'true_label':label}

            alt_im, min_eps = min_pert.evaluate(input, attack_kwargs=attack_kwargs, target=label)
            
            if alt_im is None:
                print("Could not generate adversarial example")
                advFailCount += 1
                continue

            altpred = pred(alt_im, model)
            print(f"adv_{i} is a {model.config.id2label[label.item()]} but classifies as {model.config.id2label[altpred.item()]} with epsilon {min_eps}")

            mse, ssim_val, linf = calcImageDistanceMetrics(input, alt_im)
            log.append({"index": i, "label": label.item(), "prediction": altpred.item(), "epsilon": min_eps, "mse": mse, "ssim": ssim_val, "linf": linf})

            # Save the adversarial example to new dataset
            saveImageTensor(alt_im, path_adversarial_dataset, model.config.id2label[label.item()], f"adv_{i}")
        else:
            print(f"Incorrect classification, label: {label}, prediction: {prediction}, skipping...")

    # Pickle the log
    with open(f"{path_adversarial_dataset}/log.pkl", "wb") as f:
        pickle.dump({"fails":advFailCount, "log": log}, f)

# Prepare Vim A

In [7]:
# Load Model and prepare processor

checkpointpath = "/home/jupyter/AdversarialRobustness/models/VimA.pth"
path_adversarial_dataset = '/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvVimA'
path_original_dataset = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"

model, processor = loadVim.prepareDownstreamVim()
checkpoint = torch.load(checkpointpath)
model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

dirs = sorted([d for d in os.listdir(path_original_dataset)])
id2label = {key: val for key, val in enumerate(dirs)}

model.config.id2label = id2label

genAdvUtil(model, processor, path_adversarial_dataset, path_original_dataset)


Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...


/opt/conda/envs/pytorch/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


adv_1 is a Barack Obama but classifies as Orlando Bloom with epsilon 0.004
adv_2 is a Barack Obama but classifies as George Clooney with epsilon 0.001
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
adv_5 is a Barack Obama but classifies as Leonardo DiCaprio with epsilon 0.009000000000000001
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([9], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([9], device='cuda:0'), skipping...
adv_9 is a Barack Obama but classifies as Orlando Bloom with epsilon 0.001
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='

# Prepare Vim B

In [8]:
# Load Model and prepare processor

checkpointpath = "/home/jupyter/AdversarialRobustness/models/VimB.pth"
path_adversarial_dataset = '/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvVimB'
path_original_dataset = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebVal"

model, processor = loadVim.prepareDownstreamVim()
checkpoint = torch.load(checkpointpath)
model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

dirs = sorted([d for d in os.listdir(path_original_dataset)])
id2label = {key: val for key, val in enumerate(dirs)}

model.config.id2label = id2label

genAdvUtil(model, processor, path_adversarial_dataset, path_original_dataset)



Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([8], device='cuda:0'), skipping...
adv_1 is a Barack Obama but classifies as Orlando Bloom with epsilon 0.005
adv_2 is a Barack Obama but classifies as Leonardo DiCaprio with epsilon 0.004
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([8], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
adv_5 is a Barack Obama but classifies as Orlando Bloom with epsilon 0.008
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
adv_7 is a Barack Obama but classifies as Leonardo DiCaprio with epsilon 0.001
adv_8 is a Barack Obama but classifies as Leonardo DiCaprio with epsilon 0.002
adv_9 is a Barack Obama but classifies as Orlando Bloom with epsilon 0.001
Incorrect classification, label: tensor([0], device='cuda:0'), predict

# Get Robustness data for Adversarially Trained Vim

In [7]:
# Load Model and prepare processor

checkpointpath = "/home/jupyter/AdversarialRobustness/models/AdvVimA.pth"
path_adversarial_dataset = '/home/jupyter/AdversarialRobustness/AdvRobustnessData/RobustnessVimA'
path_original_dataset = "/home/jupyter/AdversarialRobustness/CelebSubset/CelebTest"

model, processor = loadVim.prepareDownstreamVim()
checkpoint = torch.load(checkpointpath)
model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

dirs = sorted([d for d in os.listdir(path_original_dataset)])
id2label = {key: val for key, val in enumerate(dirs)}

model.config.id2label = id2label

genAdvUtil(model, processor, path_adversarial_dataset, path_original_dataset)



Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...


/opt/conda/envs/pytorch/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


adv_1 is a Barack Obama but classifies as Daniel Radcliffe with epsilon 0.002
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([2], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([2], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
Incorrect classification, label: tensor([0], device='cuda:0'), prediction: tensor([4], device='cuda:0'), skipping...
adv_9 is a Barack Obama but classifies as Daniel Radcliffe with epsilon 0.007
adv_10 is a Barack Obama 